In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import os, random, re, shutil, subprocess, sys, time

from matplotlib.pyplot import figure

from platform import python_version

print(python_version())

from scipy import stats

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#mac
util_path = "/Users/kimd999/Dropbox/script/python/srpAnalytics/code/latest/util"
sys.path.insert(0, util_path)

In [ ]:
starting_dir = os.getcwd()
print (starting_dir)

In [ ]:
# mac       - phase I, II, III - LPR - after_merging - 240 timepoints in min
complete_file_path = '/Users/kimd999/research/projects/toxicity/per_each_data/phase_I_II_III/LPR/input/phase_I_II_III_LPR_240_timepoints_morpho_filtered_min.csv'
# 383 unique chemicals, 923 unique plates

df_lpr_min = pd.read_csv(complete_file_path, header = 0)
df_lpr_min.head()
#df_LPR.shape # (72826, 29)

In [ ]:
## leave 1155 chemical only (caused a problem)
df_lpr_min_per_1155_chemical_id = df_lpr_min.loc[df_lpr_min['chemical.id'] == 1155]

plate_id_from_here = np.unique(df_lpr_min_per_1155_chemical_id['plate.id'])
for plate_id in plate_id_from_here:
    print(f"plate_id:{plate_id}\n")

    if plate_id != 13739:
        continue
    df_lpr_min_per_1155_chemical_id_plate_id = df_lpr_min_per_1155_chemical_id.loc[df_lpr_min_per_1155_chemical_id['plate.id'] == plate_id]
#        df_lpr_min_per_chemical_id_plate_id_t_only = pd.DataFrame(df_lpr_min_per_chemical_id_plate_id.iloc[:,4:28])
 
    df_lpr_min_per_1155_chemical_id_plate_id.to_csv("df_lpr_min_per_1155_chemical_id_plate_id.csv", index=False)
    
 #   print(f"df_lpr_min_per_1155_chemical_id_plate_id:{df_lpr_min_per_1155_chemical_id_plate_id}\n")

In [ ]:
## calculate MOV, AUC for all chemical concentrations
df_lpr_min_per_1155_chemical_id_plate_id = df_lpr_min_per_1155_chemical_id_plate_id.rename(columns={"plate.id": "Plate", "chemical.id": "Chemical.ID", "conc": "CONC", "well": "WELL"})
delta_mov_auc_1155 = df_lpr_min_per_1155_chemical_id_plate_id[['Chemical.ID', 'CONC', 'Plate', 'WELL']].copy()

trans_points = [2,8,14,20] # "official"

end_points = ['MOV', 'AUC']

num_light = 3 # seems reasonable since interval between middle points of each peak ~= 6
num_dark  = 3

for trans_index, trans_point in enumerate(trans_points):
#    print ("\ntrans_index:" + str(trans_index))
    print ("trans_point:" + str(trans_point))
    
    for just_index, end_point in enumerate(end_points):
        if (end_point == 'MOV'):
            delta_mov_auc_1155['MOV' + str(trans_index + 1)] \
            = df_lpr_min_per_1155_chemical_id['t' + str(trans_point + 1)] \
            - df_lpr_min_per_1155_chemical_id['t' + str(trans_point)]
        else:
            delta_mov_auc_1155['AUC' + str(trans_index + 1)] \
            = sum(df_lpr_min_per_1155_chemical_id['t' + str(trans_point + 1 + index_count)] \
                  for index_count in range(num_dark)) \
            - sum(df_lpr_min_per_1155_chemical_id['t' + str(trans_point - index_count)] \
                  for index_count in range(num_light))

display(delta_mov_auc_1155.head())
display(len(delta_mov_auc_1155))
#delta.to_csv("delta_mov_auc.csv", index=False)

In [ ]:
import generate_dose_response as gdr
import BMD_BMDL_estimation as bmdest
import Plot_Save as ps

In [ ]:
## calculate BMD
start_time = time.time()
os.chdir(starting_dir)

if (os.path.isdir("output") == True):
    shutil.rmtree("output")
os.mkdir("output")

output_folder = os.path.join(starting_dir, "output")
os.chdir(output_folder)

#full_devel = "full"
full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(delta_mov_auc_1155['Chemical.ID'])
    end_points_from_here = ['MOV1','AUC1']
else:
    chemical_id_from_here = [1155]
    end_points_from_here = ['AUC1']

report = True
#report = False

for chemical_id in chemical_id_from_here:
    if (report): print("chemical_id:" + str(chemical_id))
    for end_point in end_points_from_here:
        if (report): print("end_point:" + str(end_point))
        # subset original dataframe for a user-specified chemical and end_point pair
        delta_mov_auc_end_point_chemical_id \
        = delta_mov_auc_1155.loc[delta_mov_auc_1155['Chemical.ID'] == chemical_id,['Chemical.ID', 'CONC', 'Plate', 'WELL', end_point]]
        
        delta_mov_auc_end_point_chemical_id.to_csv("delta_mov_auc_end_point_chemical_id.csv", index=False)
        print(f"delta_mov_auc_end_point_chemical_id:\n{delta_mov_auc_end_point_chemical_id}")
        #print("type(delta_mov_auc_end_point_chemical_id):\n"+str(type(delta_mov_auc_end_point_chemical_id)))
        #print("type(end_point):\n"+str(type(end_point)))

        dose_response = gdr.gen_dose_response_behavior(delta_mov_auc_end_point_chemical_id, end_point)
        if (report): print(f"dose_response:{dose_response}\n")
        qc_flag = gdr.BMD_feasibility_analysis(dose_response)
        test_dose_response = gdr.reformat_dose_response(dose_response)
        #test_dose_response = dose_response
        if(qc_flag in [0, 1]):
            # No BMD analysis required. Generate report and exit
            ps.save_results_poor_data_or_no_convergence(test_dose_response, qc_flag, str(chemical_id), end_point, None)
        else:
            # Fit dose response models
            model_predictions = bmdest.analyze_dose_response_data(test_dose_response)
            # Select best model
            selected_model_params = bmdest.select_model(model_predictions)
            # Check if unique model is found
            unique_model_flag = selected_model_params['no_unique_model_found_flag']
            if(unique_model_flag == 0):
                # Generate report
                ps.save_results_good_data_unique_model(test_dose_response, qc_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
            else:
                bmd_analysis_flag = selected_model_params['model_select_flag']
                if(bmd_analysis_flag == 1):
                    print(f"test_dose_response:\n{test_dose_response}")
                    ps.save_results_poor_data_or_no_convergence(test_dose_response, qc_flag, str(chemical_id), end_point, selected_model_params)
                else:
                    ps.save_results_good_data_nounique_model(test_dose_response, qc_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Done, it took:"+str(time_took))
# 1 chemical (3756) and 2 endpoints (['MOV1','AUC1']), 140 seconds took
# 7 chemicals and 2 endpoints (['MOV1','AUC1']), 6 minutes took

time_filename = 'running_time.txt'
f_time = open(time_filename, 'w')
f_time.write(str(time_took))
f_time.close()

In [6]:
## leave 1016 chemical only (ok)
df_lpr_min_per_1016_chemical_id = df_lpr_min.loc[df_lpr_min['chemical.id'] == 1016]

plate_id_from_here = np.unique(df_lpr_min_per_1016_chemical_id['plate.id'])
for plate_id in plate_id_from_here:
    print(f"plate_id:{plate_id}\n")

    df_lpr_min_per_1016_chemical_id_plate_id = df_lpr_min_per_1016_chemical_id.loc[df_lpr_min_per_1016_chemical_id['plate.id'] == plate_id]
#        df_lpr_min_per_chemical_id_plate_id_t_only = pd.DataFrame(df_lpr_min_per_chemical_id_plate_id.iloc[:,4:28])

    #print(f"df_lpr_min_per_1016_chemical_id_plate_id:{df_lpr_min_per_1016_chemical_id_plate_id}\n")


plate_id:22200

df_lpr_min_per_1016_chemical_id_plate_id:       chemical.id   conc  plate.id well         t0      t1      t2  \
37900         1016  0.000     22200  A02   5.551000   3.520   9.753   
37901         1016  0.000     22200  A03   0.000000   0.000   0.000   
37902         1016  0.000     22200  A04   1.636000   0.958   6.570   
37903         1016  0.000     22200  A05  61.335001  16.367  18.387   
37904         1016  0.000     22200  A07  12.575000  15.743  20.883   
...            ...    ...       ...  ...        ...     ...     ...   
37972         1016  0.410     22200  H07   0.000000   0.000   0.000   
37973         1016  0.140     22200  H08   0.000000   0.046   3.539   
37974         1016  0.046     22200  H09   6.241000  14.917   3.778   
37975         1016  0.015     22200  H10   4.467000   3.781  16.082   
37976         1016  0.000     22200  H12   0.000000   2.829   9.334   

               t3          t4      t5  ...        t14         t15         t16  \
37900  17

In [7]:
## calculate MOV, AUC for all chemical concentrations
df_lpr_min_per_1016_chemical_id = df_lpr_min_per_1016_chemical_id.rename(columns={"plate.id": "Plate", "chemical.id": "Chemical.ID", "conc": "CONC", "well": "WELL"})
delta_mov_auc_1016 = df_lpr_min_per_1016_chemical_id[['Chemical.ID', 'CONC', 'Plate', 'WELL']].copy()
#delta_mov_auc = df_lpr_min[['chemical.id', 'conc', 'plate.id', 'well']].copy()

trans_points = [2,8,14,20] # "official"

end_points = ['MOV', 'AUC']

num_light = 3 # seems reasonable since interval between middle points of each peak ~= 6
num_dark  = 3

for trans_index, trans_point in enumerate(trans_points):
#    print ("\ntrans_index:" + str(trans_index))
    print ("trans_point:" + str(trans_point))
    
    for just_index, end_point in enumerate(end_points):
        if (end_point == 'MOV'):
            delta_mov_auc_1016['MOV' + str(trans_index + 1)] \
            = df_lpr_min_per_1016_chemical_id['t' + str(trans_point + 1)] \
            - df_lpr_min_per_1016_chemical_id['t' + str(trans_point)]
        else:
            delta_mov_auc_1016['AUC' + str(trans_index + 1)] \
            = sum(df_lpr_min_per_1016_chemical_id['t' + str(trans_point + 1 + index_count)] \
                  for index_count in range(num_dark)) \
            - sum(df_lpr_min_per_1016_chemical_id['t' + str(trans_point - index_count)] \
                  for index_count in range(num_light))

#display(delta_mov_auc.head(1))
#delta.to_csv("delta_mov_auc.csv", index=False)

trans_point:2
trans_point:8
trans_point:14
trans_point:20


In [8]:
start_time = time.time()
os.chdir(starting_dir)

if (os.path.isdir("output") == True):
    shutil.rmtree("output")
os.mkdir("output")

output_folder = os.path.join(starting_dir, "output")
os.chdir(output_folder)

full_devel = "full"
#full_devel = "devel"

if (full_devel == "full"):
    chemical_id_from_here = np.unique(delta_mov_auc_1016['Chemical.ID'])
    end_points_from_here = ['MOV1','AUC1']
else:
    chemical_id_from_here = [53]
    end_points_from_here = ['MOV1']

#report = True
report = False

for chemical_id in chemical_id_from_here:
    if (report): print("chemical_id:" + str(chemical_id))
    for end_point in end_points_from_here:
        if (report): print("end_point:" + str(end_point))
        # subset original dataframe for a user-specified chemical and end_point pair
        delta_mov_auc_end_point_chemical_id \
        = delta_mov_auc_1016.loc[delta_mov_auc_1016['Chemical.ID'] == chemical_id,['Chemical.ID', 'CONC', 'Plate', 'WELL', end_point]]
        #print("delta_mov_auc_end_point_chemical_id:\n"+str(delta_mov_auc_end_point_chemical_id))
        #print("type(delta_mov_auc_end_point_chemical_id):\n"+str(type(delta_mov_auc_end_point_chemical_id)))
        #print("type(end_point):\n"+str(type(end_point)))

        dose_response = gdr.gen_dose_response_behavior(delta_mov_auc_end_point_chemical_id, end_point)
        if (report): print("dose_response:\n"+str(dose_response))
        qc_flag = gdr.BMD_feasibility_analysis(dose_response)
        test_dose_response = gdr.reformat_dose_response(dose_response)
        #test_dose_response = dose_response
        if(qc_flag in [0, 1]):
            # No BMD analysis required. Generate report and exit
            ps.save_results_poor_data_or_no_convergence(test_dose_response, qc_flag, str(chemical_id), end_point, None)
        else:
            # Fit dose response models
            model_predictions = bmdest.analyze_dose_response_data(test_dose_response)
            # Select best model
            selected_model_params = bmdest.select_model(model_predictions)
            # Check if unique model is found
            unique_model_flag = selected_model_params['no_unique_model_found_flag']
            if(unique_model_flag == 0):
                # Generate report
                ps.save_results_good_data_unique_model(test_dose_response, qc_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
            else:
                bmd_analysis_flag = selected_model_params['model_select_flag']
                if(bmd_analysis_flag == 1):
                    ps.save_results_poor_data_or_no_convergence(test_dose_response, qc_flag, str(chemical_id), end_point, selected_model_params)
                else:
                    ps.save_results_good_data_nounique_model(test_dose_response, qc_flag, model_predictions, selected_model_params, str(chemical_id), end_point)
end_time = time.time()
time_took = str(round((end_time-start_time), 1)) + " seconds"
print ("Done, it took:"+str(time_took))
# 1 chemical (3756) and 2 endpoints (['MOV1','AUC1']), 140 seconds took
# 7 chemicals and 2 endpoints (['MOV1','AUC1']), 6 minutes took

time_filename = 'running_time.txt'
f_time = open(time_filename, 'w')
f_time.write(str(time_took))
f_time.close()



hyper_response_vals:
Series([], Name: MOV1, dtype: float64)


type(hyper_response_vals):
<class 'pandas.core.series.Series'>
hypo_response_vals:
37929    0.0
Name: MOV1, dtype: float64


hyper_response_vals:
38015    171.602001
Name: MOV1, dtype: float64


type(hyper_response_vals):
<class 'pandas.core.series.Series'>
hypo_response_vals:
38034   -103.635002
38055     -0.713000
Name: MOV1, dtype: float64


hyper_response_vals:
Series([], Name: MOV1, dtype: float64)


type(hyper_response_vals):
<class 'pandas.core.series.Series'>
hypo_response_vals:
37918   -5.505
37954   -1.853
Name: MOV1, dtype: float64


hyper_response_vals:
Series([], Name: MOV1, dtype: float64)


type(hyper_response_vals):
<class 'pandas.core.series.Series'>
hypo_response_vals:
38033   -28.444
38054     0.000
Name: MOV1, dtype: float64


hyper_response_vals:
37936    142.474999
Name: MOV1, dtype: float64


type(hyper_response_vals):
<class 'pandas.core.series.Series'>
hypo_response_vals:
Series([], Name: MOV1, dty